In [51]:
from keras.preprocessing.image import ImageDataGenerator

In [52]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [53]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='WImD1lYFgLr7ETzVUrUheKzW-cdRHnZBTZU5S49O9gq3',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageclassification-donotdelete-pr-ohyztlr8kisyqz'
object_key = 'Nutrition classifier_zipped.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [54]:
if not hasattr(streaming_body_1,"__iter__"):streaming_body_1.__iter__ = types.MethodType(__iter__,streaming_body_1)

In [55]:
from io import BytesIO
from zipfile import ZipFile
unzip = ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [56]:
import os 
filenames = os.listdir('/home/wsuser/work/Nutrition classifier/TEST_SET')

In [57]:
x_train=train_datagen.flow_from_directory(r'/home/wsuser/work/Nutrition classifier/TRAIN_SET',target_size=(64,64),batch_size=5,color_mode='rgb',class_mode='sparse')
x_test = train_datagen.flow_from_directory(r'/home/wsuser/work/Nutrition classifier/TEST_SET',target_size=(64,64),batch_size=5,color_mode='rgb',class_mode='sparse')


Found 2626 images belonging to 5 classes.
Found 1055 images belonging to 5 classes.


In [58]:
print(x_test.class_indices)

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [59]:
from collections import Counter as c
c(x_train.labels)

Counter({0: 606, 1: 445, 2: 479, 3: 621, 4: 475})

In [60]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout
from keras.preprocessing.image import ImageDataGenerator

In [61]:
model=Sequential()
classifier=Sequential()

In [62]:
classifier.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Conv2D(32,(3,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Flatten())
classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dense(units=5,activation='softmax'))

In [63]:
classifier.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_2 (Dense)             (None, 128)              

In [64]:
classifier.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [65]:
classifier.fit_generator(generator=x_train,steps_per_epoch=len(x_train),epochs= 2 ,validation_data=x_test,validation_steps=len(x_test))

/tmp/wsuser/ipykernel_164/3830403452.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(generator=x_train,steps_per_epoch=len(x_train),epochs= 2 ,validation_data=x_test,validation_steps=len(x_test))


Epoch 1/2
526/526 [==============================] - 32s 59ms/step - loss: 0.1204 - accuracy: 0.9539 - val_loss: 0.0825 - val_accuracy: 0.9611
Epoch 2/2
526/526 [==============================] - 32s 61ms/step - loss: 0.0455 - accuracy: 0.9863 - val_loss: 0.0702 - val_accuracy: 0.9754


In [17]:
classifier.save('/home/wsuser/work/nutrition.h5')

In [18]:
!tar -zcvf image-classification-model_new.tgz nutrition.h5

nutrition.h5


In [19]:
ls -1

image-classification-model_new.tgz
'Nutrition classifier'/
nutrition.h5


In [20]:
! pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 13.2 MB/s eta 0:00:01


In [21]:
from ibm_watson_machine_learning import APIClient
wml_credintials = {
    "url" : "https://us-south.ml.cloud.ibm.com",
    "apikey" : "3EMrL-7wESBZLFCk0abidBdj4Pnlz-7Hsiqd3E39NCQX"
}

client = APIClient(wml_credintials)

In [22]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [23]:
space_uid = guid_from_space_name(client , 'imageclassification')
print("Space UID = "+ space_uid)

Space UID = 68f147b7-3c13-4157-be5a-85a5c50f29ee


In [24]:
client.set.default_space(space_uid)

'SUCCESS'

In [25]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [26]:
client.software_specifications.list(limit = 100)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1

In [27]:
model_details = client.repository.store_model(model = 'image-classification-model_new.tgz' , meta_props = {
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE : "tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID : software_spec_uid
})

model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [28]:
model_id

'bbd12864-53fb-4b64-ab1f-11d45246f865'

In [29]:
client.repository.download(model_id, 'nutrition_analyzer2_model.h5')

Successfully saved model content to file: 'nutrition_analyzer2_model.h5'


'/home/wsuser/work/nutrition_analyzer2_model.h5'